# Clothing Material and Heat Transfer

Kristtiya Guerra and Audrey Lee

In [31]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

### The Human Body



I will set a `State` object for the initial temperature of the human body (in Kelvin).

In [ ]:
meter = UNITS.meter
watt = UNITS.watt
minute = UNITS.minute
kelvin = UNITS.kelvin
celsius = UNITS.celsius

HumanTemp = 37 + 273.15 
WinterInit = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.41/1.136, bottom=0.21/1.136, socks=0.04/1.136, 
                   Atop=1.6*.47, Abottom=1.6*.45, Asocks=1.6*.07, rHuman=.003874)

meter = UNITS.meter
watt = UNITS.watt
minute = UNITS.minute
kelvin = UNITS.kelvin
celsius = UNITS.celsius

HumanTemp = 37 + 273.15 
WinterInit = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.41/1.136, bottom=0.21/1.136, socks=0.04/1.136, 
                   Atop=1.6*.47, Abottom=1.6*.45, Asocks=1.6*.07, rHuman=.003874)

SummerInit = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.07/1.136, bottom=0.09/1.136, socks=0.03/1.136, 
                   Atop=1.6*.12, Abottom=1.6*.18, Asocks=1.6*.05)

LoungeInit = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.38/1.136, bottom=0.34/1.136, socks=0.03/1.136, 
                   Atop=1.6*.45, Abottom=1.6*.44, Asocks=1.6*.07)

DressInit = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.83/1.136, bottom=0.59/1.136, socks=0.02/1.136, 
                   Atop=1.6*.40, Abottom=1.6*.19, Asocks=1.6*.51)
Nothing = State(HumanTemp=HumanTemp, dQdt = 0, 
                   top =0.0000001/1.136, bottom=0.0000001/1.136, socks=0.0000001/1.136, 
                   Atop=1.6, Abottom=1.6, Asocks=1.6)

And a `System` object to contain the system parameters.

In [37]:
human = System(WinterInit=WinterInit,T_env= 5+273.15,
               t_end=0.02,
               dt=0.0007)

,values
WinterInit,HumanTemp 310.150000 dQdt 0.00000...
T_env,278.15
t_end,0.02
dt,0.0007


human = System(WinterInit=WinterInit, 
               SummerInit=SummerInit,
               LoungeInit=LoungeInit,
               DressInit=DressInit,
               Nothing=Nothing,
               T_env= 5+273.15,
               t_end=0.02,
               dt=0.0007)

In [38]:
def Clothes(state, t, system):
    """Update the thermal transfer model.
    
    state: State (temp)
    t: time
    system: System object
    
    returns: State (temp)
    """
    unpack(system)
    
    HumanTemp, dQdt, mass1, mass2, mass3, specHeat1, specHeat2, specHeat3, top, bottom, socks, Atop, Abottom, Asocks, rHuman = state
    
    T_delta = HumanTemp - T_env
    dQdt = (Atop * (T_delta)/top) + (Abottom * (T_delta)/bottom) + (Asocks * (T_delta)/socks) + ((1-(Atop+Abottom+Asocks)) * T_delta/rHuman)
    dTdt = dQdt / ((mass1+mass2+mass3) * (specHeat1+specHeat2+specHeat3))
    HumanTemp -= (dTdt) * dt
    
    return State(HumanTemp=HumanTemp, dQdt=dQdt, mass1=mass1, mass2=mass2, mass3=mass3, 
                 specHeat1=specHeat1, specHeat2=specHeat2, specHeat3=specHeat3, 
                 top=top, bottom=bottom, socks=socks, Atop=Atop, Abottom=Abottom, Asocks=Asocks)

In [39]:
def run_simulation(system, Clothes):
    """Runs a simulation of the system.
    
    Add a TimeFrame to the System: results
    
    system: System object
    update_func: function that updates state
    """
    unpack(system)
    
    frame = TimeFrame(columns=WinterInit.index)
    frame.row[0] = WinterInit
    ts = linrange(0, t_end, dt)
    
    for t in ts:
        frame.row[t+dt] = Clothes(frame.row[t], t, system)

        
    # store the final temperature in T_final
    system.T_final = get_last_value(frame.HumanTemp)
    
    return frame

def run_simulation(system, Clothes):
    """Runs a simulation of the system.
    
    Add a TimeFrame to the System: results
    
    system: System object
    update_func: function that updates state
    """
    unpack(system)
    
    frame = TimeFrame(columns=WinterInit.index)
    frame2 = TimeFrame(columns=SummerInit.index)
    frame3=TimeFrame(columns=LoungeInit.index)
    frame4=TimeFrame(columns=DressInit.index)
    frame5=TimeFrame(columns=Nothing.index)
    
    frame.row[0] = WinterInit
    frame2.row[0] = SummerInit
    frame3.row[0] = LoungeInit
    frame4.row[0] = DressInit
    frame5.row[0] = Nothing
    
    ts = linrange(0, t_end, dt)
    
    for t in ts:
        frame.row[t+dt] = Clothes(frame.row[t], t, system)
        frame2.row[t+dt] = Clothes(frame2.row[t],t,system)
        frame3.row[t+dt] = Clothes(frame3.row[t],t,system)
        frame4.row[t+dt] = Clothes(frame4.row[t],t,system)
        frame5.row[t+dt] = Clothes(frame5.row[t],t,system)
        
    # store the final temperature in T_final
    system.T_final = get_last_value(frame.HumanTemp)
    system.T_final2 = get_last_value(frame2.HumanTemp)
    system.T_final3 = get_last_value(frame3.HumanTemp)
    system.T_final4 = get_last_value(frame4.HumanTemp)
    system.T_final5 = get_last_value(frame5.HumanTemp)
    
    return frame, frame2, frame3, frame4, frame5

In [40]:
results = run_simulation(human, Clothes)

ValueError: too many values to unpack (expected 8)

In [14]:
plot(results.HumanTemp, label='Winter Clothes')
# plot(results[1].HumanTemp, label='Summer Clothes')
# plot(results[2].HumanTemp, label='Lounge Clothes')
# plot(results[3].HumanTemp, label='Dress Clothes')
# plot(results[4].HumanTemp, label='Nothing')

decorate(xlabel='Time (day)',
         ylabel='Temperature (K)')

NameError: name 'results' is not defined

In [15]:
human.T_final

AttributeError: 'System' object has no attribute 'T_final'

In [63]:
human.T_final2

AttributeError: 'System' object has no attribute 'T_final2'

In [64]:
human.T_final3

AttributeError: 'System' object has no attribute 'T_final3'

In [41]:
human.T_final4

290.074307140556

In [42]:
human.Q_final5

-2.369357890044128e+134